In [1]:
import numpy as np
from scipy.stats import norm
from scipy import integrate
import torch

from bo_functions import ei_approx_ln_term
from bo_functions import calc_ei_emulator

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
error_best = 1.0376255530659095
pred_mean = np.array([-8.14817608, -1.95393033,  0.00769104701, -2.03767010, -2.11993305])
pred_stdev = np.array([1.58153871, 1.52678842, 1.52182998, 1.52678842, 1.58153871])
pred_var = np.array([1.58153871, 1.52678842, 1.52182998, 1.52678842, 1.58153871])**2
y_target = np.array([-14.0031178, -2.99270996,  0.00217820788,  0.991009082, 5.97513219])
explore_bias = torch.tensor([1])

In [5]:
error_best = 1.0376255530659095
pred_mean = np.array([-8.14817608, -1.95393033,  0.00769104701, -2.03767010, -2.11993305])
pred_stdev = np.array([1.58153871, 1.52678842, 1.52182998, 1.52678842, 1.58153871])
pred_var = np.array([1.58153871, 1.52678842, 1.52182998, 1.52678842, 1.58153871])**2
y_target = np.array([-14.0031178, -2.99270996,  0.00217820788,  0.991009082, 5.97513219])
explore_bias = torch.tensor([1])

In [6]:
%%time
ei = 0

for i in range(len(y_target)):
    with np.errstate(divide = 'warn'):
        bound_a = ((y_target[i] - pred_mean[i]) + np.sqrt(np.exp(error_best*explore_bias)))/pred_stdev[i] #1xn
        bound_b = ((y_target[i] - pred_mean[i]) - np.sqrt(np.exp(error_best*explore_bias)))/pred_stdev[i] #1xn
        bound_lower = np.min([bound_a,bound_b])
        bound_upper = np.max([bound_a,bound_b])
#         print(bound_lower,bound_upper)

        args = (error_best, pred_mean[i], pred_stdev[i], y_target[i], explore_bias)

        ei_term_1 = (error_best*explore_bias)*( norm.cdf(bound_upper)-norm.cdf(bound_lower) )
    #         print(ei_term_1)
        ei_term_2_out = integrate.quad(ei_approx_ln_term, bound_lower, bound_upper, args = args, full_output = 1)
        ei_term_2 = (-1)*ei_term_2_out[0] 
#         print(ei_term_1, ei_term_2)
        term_2_abs_err = ei_term_2_out[1]
        EI = ei_term_1 + ei_term_2
        ei += EI

print(ei)

#Note: 1min + 45sec per iteration

tensor([0.0043], dtype=torch.float64) -0.0006488282026193835
tensor([0.6488], dtype=torch.float64) 0.6956272650523775
tensor([0.7579], dtype=torch.float64) 0.89634916087324
tensor([0.1946], dtype=torch.float64) 0.09623339678034726
tensor([2.5877e-05], dtype=torch.float64) -1.1579830923517208e-05
tensor([3.2931], dtype=torch.float64)
CPU times: user 336 ms, sys: 9.74 ms, total: 345 ms
Wall time: 333 ms


In [4]:
%%time
ei = 0

for i in range(len(y_target)):
    ei_x = calc_ei_emulator(error_best,pred_mean[i],pred_var[i],y_target[i], explore_bias, obj = "LN_obj")
    ei += ei_x
    
print(ei)

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creatin

tensor([3.2931], dtype=torch.float64)
CPU times: user 316 ms, sys: 8.63 ms, total: 325 ms
Wall time: 320 ms
